In [64]:
# Collect desired calendar properties from initial input
# 1- access data source
# 2- creat calendar structure for each calendar
# 3- Fill structure with calendar properties and calendar data
# 4- Process calendar data

In [65]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [66]:
# specify the file path, you might need to adjust this
file_path = "HousingSTP-Update.xlsx"
# Load the calendar data into a DataFrame
calendar_df = pd.read_excel(file_path, sheet_name='CALENDAR')
# Load a sheet into a dataframe by its name
project_df = pd.read_excel(file_path, sheet_name='PROJECT')

In [67]:
calendar_df

,%F,clndr_id,default_flag,clndr_name,proj_id,base_clndr_id,last_chng_date,clndr_type,day_hr_cnt,week_hr_cnt,month_hr_cnt,year_hr_cnt,rsrc_private,clndr_data
0,%R,5829,N,Trades - 5 Day Workweek,NaN,NaN,2000-04-26 14:11,CA_Base,8,40,172,2000,N,(0||CalendarData()((0||DaysOfWeek()((0||1()())...
1,%R,7219,N,TRAIL RUN NEW,NaN,NaN,2024-08-07 00:00,CA_Base,8,48,192,2304,N,(0||CalendarData()( (0||DaysOfWeek()( ...
2,%R,8467,Y,New Scope Arar Project,NaN,NaN,2024-10-03 00:00,CA_Base,8,40,172,2000,N,(0||CalendarData()( (0||DaysOfWeek()( ...


In [68]:
calendar_struct = calendar_df.iloc[2].to_dict()
# get first key
first_key = next(iter(calendar_struct))

# delete first key-value pair
calendar_struct.pop(first_key)

'%R'

In [69]:
calendar_struct

{'clndr_id': 8467,
 'default_flag': 'Y',
 'clndr_name': 'New Scope Arar Project',
 'proj_id': nan,
 'base_clndr_id': nan,
 'last_chng_date': '2024-10-03 00:00',
 'clndr_type': 'CA_Base',
 'day_hr_cnt': 8,
 'week_hr_cnt': 40,
 'month_hr_cnt': 172,
 'year_hr_cnt': 2000,
 'rsrc_private': 'N',
 'clndr_data': '(0||CalendarData()(\x7f\x7f  (0||DaysOfWeek()(\x7f\x7f    (0||1()(\x7f\x7f      (0||0(s|08:00|f|16:00)())))\x7f\x7f    (0||2()(\x7f\x7f      (0||0(s|08:00|f|16:00)())))\x7f\x7f    (0||3()(\x7f\x7f      (0||0(s|08:00|f|16:00)())))\x7f\x7f    (0||4()(\x7f\x7f      (0||0(s|08:00|f|16:00)())))\x7f\x7f    (0||5()(\x7f\x7f      (0||0(s|08:00|f|16:00)())))\x7f\x7f    (0||6()())\x7f\x7f    (0||7()(\x7f\x7f      (0||0(s|08:00|f|16:00)())))))\x7f\x7f  (0||VIEW(ShowTotal|Y)())\x7f\x7f  (0||Exceptions()(\x7f\x7f    (0||0(d|40179)())\x7f\x7f    (0||1(d|40329)())\x7f\x7f    (0||2(d|40364)())\x7f\x7f    (0||3(d|40427)())\x7f\x7f    (0||4(d|40507)())\x7f\x7f    (0||5(d|40536)())\x7f\x7f    (0||6(d|40

In [70]:
# Assuming 'clndr_data' is a column in 'calendar_df'
clndr_data = pd.Series(calendar_struct['clndr_data'])
clndr_data_split = clndr_data.str.split('\\(0\\|\\|', expand=True).stack()

# Create new dataframe with each output of split operation as a new row
cal_data_df = pd.DataFrame(clndr_data_split, columns=['Data'])

# Reset the index
cal_data_df = cal_data_df.reset_index(drop=True)

In [71]:
cal_data_df

,Data
0,
1,CalendarData()(
2,DaysOfWeek()(
3,1()(
4,0(s|08:00|f|16:00)())))
5,2()(
6,0(s|08:00|f|16:00)())))
7,3()(
8,0(s|08:00|f|16:00)())))
9,4()(


In [72]:
start_index = cal_data_df[cal_data_df['Data'].str.contains('DaysOfWeek')].index[0]
end_index = cal_data_df[cal_data_df['Data'].str.contains('VIEW')].index[0]

# Create a new DataFrame by selecting rows between "DaysOfWeek" and "VIEW"
DaysOfWeek_df = cal_data_df.loc[start_index + 1: end_index - 1]

In [73]:
# Initialize a list to hold the data
WeekDays_data = []

# Iterate over rows in DaysOfWeek_df
for i in range(len(DaysOfWeek_df['Data'])):

    if DaysOfWeek_df['Data'].iloc[i][0] != '0':
        # Add new entry and mark it as the current entry
        current_entry = [DaysOfWeek_df['Data'].iloc[i][0], '0', '0']
        WeekDays_data.append(current_entry)

    elif 's' in DaysOfWeek_df['Data'].iloc[i] and 'f' in DaysOfWeek_df['Data'].iloc[i]:
        # Modify 'StartingHour' and 'EndingHour' of current entry
        start_s = DaysOfWeek_df['Data'].iloc[i].index('s') + 2
        start_f = DaysOfWeek_df['Data'].iloc[i].index('f') + 2
        current_entry[1] = DaysOfWeek_df['Data'].iloc[i][start_s: start_s + 5]
        current_entry[2] = DaysOfWeek_df['Data'].iloc[i][start_f: start_f + 5]

# Create DataFrame
WeekDays = pd.DataFrame(WeekDays_data, columns=['DayNumber', 'StartingHour', 'EndingHour'])

In [74]:
WeekDays

,DayNumber,StartingHour,EndingHour
0,1,08:00,16:00
1,2,08:00,16:00
2,3,08:00,16:00
3,4,08:00,16:00
4,5,08:00,16:00
5,6,0,0
6,7,08:00,16:00


In [75]:
# Get index of row containing 'Exceptions' in 'Data' column
exceptions_index = cal_data_df[cal_data_df['Data'].str.contains('Exceptions')].index[0]

# Select all rows after that index
exceptions_df = cal_data_df.iloc[exceptions_index + 1:]

In [76]:
# Initialize a list to hold the data
Exceptions_data = []

# Iterate over rows in exceptions_df
for i in range(len(exceptions_df['Data'])):

    if 'd' in exceptions_df['Data'].iloc[i]:
        # Add new entry and mark it as the current entry
        idx = exceptions_df['Data'].iloc[i].index('d')
        current_entry = [exceptions_df['Data'].iloc[i][idx + 2:idx + 7], '0', '0']
        Exceptions_data.append(current_entry)

        # If next row contains 's' and 'f', modify 'StartingHour' and 'EndingHour' of current entry
        if i < len(exceptions_df['Data']) - 1 and 's' in exceptions_df['Data'].iloc[i + 1] and 'f' in \
                exceptions_df['Data'].iloc[i + 1]:
            start_s = exceptions_df['Data'].iloc[i + 1].index('s') + 2
            start_f = exceptions_df['Data'].iloc[i + 1].index('f') + 2
            current_entry[1] = exceptions_df['Data'].iloc[i + 1][start_s: start_s + 5]
            current_entry[2] = exceptions_df['Data'].iloc[i + 1][start_f: start_f + 5]

# Create DataFrame
Exceptions = pd.DataFrame(Exceptions_data, columns=['DayNumber', 'StartingHour', 'EndingHour'])

In [77]:
Exceptions

,DayNumber,StartingHour,EndingHour
0,40179,0,0
1,40329,0,0
2,40364,0,0
3,40427,0,0
4,40507,0,0
5,40536,0,0
6,40543,0,0
7,40693,0,0
8,40728,0,0
9,40791,0,0


In [78]:
project_df

,%F,proj_id,fy_start_month_num,rsrc_self_add_flag,allow_complete_flag,rsrc_multi_assign_flag,checkout_flag,project_flag,step_complete_flag,cost_qty_recalc_flag,batch_sum_flag,name_sep_char,def_complete_pct_type,proj_short_name,acct_id,orig_proj_id,source_proj_id,base_type_id,clndr_id,sum_base_proj_id,task_code_base,task_code_step,priority_num,wbs_max_sum_level,strgy_priority_num,last_checksum,critical_drtn_hr_cnt,def_cost_per_qty,last_recalc_date,plan_start_date,plan_end_date,scd_end_date,add_date,last_tasksum_date,fcst_start_date,def_duration_type,task_code_prefix,guid,def_qty_type,add_by_name,web_local_root_path,proj_url,def_rate_type,add_act_remain_flag,act_this_per_link_flag,def_task_type,act_pct_link_flag,critical_path_type,task_code_prefix_flag,def_rollup_dates_flag,use_project_baseline_flag,rem_target_link_flag,reset_planned_flag,allow_neg_act_flag,sum_assign_level,last_fin_dates_id,fintmpl_id,last_baseline_update_date,cr_external_key,apply_actuals_date,location_id,last_schedule_date,loaded_scope_level,export_flag,new_fin_dates_id,baselines_to_export,baseline_names_to_export,next_data_date,close_period_flag,sum_refresh_date,trsrcsum_loaded,sumtask_loaded
0,%R,8639,1,Y,Y,Y,N,Y,N,N,Y,.,CP_Phys,2107RV01RC01171024,NaN,NaN,NaN,NaN,8467,8640,1000,10,10,2,500,NaN,0,0.0,2024-10-17 16:00,2023-03-01 00:00,NaN,2025-10-22 13:00,2024-10-16 11:12,NaN,NaN,DT_FixedDUR2,A,x3eepFCZSUeynSA/WWglJw,QT_Hour,ADMIN,NaN,NaN,COST_PER_QTY,N,Y,TT_Task,Y,CT_TotFloat,Y,Y,Y,Y,N,N,SL_Taskrsrc,NaN,1,NaN,NaN,NaN,NaN,2024-10-19 10:20,7,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [79]:
import pandas as pd
from datetime import datetime, timedelta

# This function converts Excel's date serial number to python datetime
def excel_date_to_dt(date):
    return (pd.to_datetime('1900-01-01') + pd.to_timedelta(date - 2, unit='d')).date()

# Find the maximum DayNumber in the Exceptions dataframe 
max_daynumber = Exceptions['DayNumber'].max()

# Convert max_daynumber to integer if it's not already
max_daynumber = int(max_daynumber)

# Convert the max day number to a date
max_daynumber_date = excel_date_to_dt(max_daynumber)

# Get the first scd_end_date from the project_df dataframe
scd_end_date = project_df['scd_end_date'].iloc[0]

# Make sure scd_end_date is in a date format
scd_end_date = pd.to_datetime(scd_end_date).date()

# Compare the date from max_daynumber_date and scd_end_date, take the later one
if max_daynumber_date > scd_end_date:
    project_finish_date = max_daynumber_date
else:
    project_finish_date = scd_end_date

In [80]:
project_start_date = pd.to_datetime(project_df['plan_start_date']).iloc[0]
#project_finish_date = pd.to_datetime(project_df['scd_end_date']).iloc[0]

In [81]:
project_start_date

Timestamp('2023-03-01 00:00:00')

In [82]:
project_finish_date

datetime.date(2025, 10, 22)

In [83]:
start_date = project_start_date.date()

In [84]:
start_date

datetime.date(2023, 3, 1)

In [85]:
WeekDays

,DayNumber,StartingHour,EndingHour
0,1,08:00,16:00
1,2,08:00,16:00
2,3,08:00,16:00
3,4,08:00,16:00
4,5,08:00,16:00
5,6,0,0
6,7,08:00,16:00


In [86]:
# Reindexing DayNumber in WeekDays to match Python's date convention Monday=0, Sunday=6
mapping = {1:6, 2:0, 3:1, 4:2, 5:3, 6:4, 7:5}
WeekDays['DayNumber'] = WeekDays['DayNumber'].astype(int)
WeekDays['DayNumber'] = WeekDays['DayNumber'].map(mapping)

In [87]:
WeekDays

,DayNumber,StartingHour,EndingHour
0,6,08:00,16:00
1,0,08:00,16:00
2,1,08:00,16:00
3,2,08:00,16:00
4,3,08:00,16:00
5,4,0,0
6,5,08:00,16:00


In [88]:
# creating date range
Calendar_df = pd.DataFrame({'Date': pd.date_range(start=project_start_date, end=project_finish_date)})

# formatting date to dd-mm-yyyy format
Calendar_df['Date'] = Calendar_df['Date'].dt.strftime('%d-%m-%Y')

# getting the weekday number where Monday=0, Sunday=6
Calendar_df['DayNumber'] = pd.to_datetime(Calendar_df['Date'], format='%d-%m-%Y').dt.dayofweek

# merging Calendar_df with WeekDays on 'DayNumber' to get 'StartingHour' and 'EndingHour'
Calendar_df = pd.merge(Calendar_df, WeekDays, on='DayNumber', how='left')

# removing the 'DayNumber' column as it's no longer needed
Calendar_df = Calendar_df.drop(columns='DayNumber')

In [89]:
Calendar_df

,Date,StartingHour,EndingHour
0,01-03-2023,08:00,16:00
1,02-03-2023,08:00,16:00
2,03-03-2023,0,0
3,04-03-2023,08:00,16:00
4,05-03-2023,08:00,16:00
5,06-03-2023,08:00,16:00
6,07-03-2023,08:00,16:00
7,08-03-2023,08:00,16:00
8,09-03-2023,08:00,16:00
9,10-03-2023,0,0


In [90]:
print(Calendar_df.dtypes)

Date            object
StartingHour    object
EndingHour      object
dtype: object


In [91]:
# Fixing the "0" in hours column
Calendar_df['StartingHour'] = Calendar_df['StartingHour'].replace({"0": "00:00"})
Calendar_df['EndingHour'] = Calendar_df['EndingHour'].replace({"0": "00:00"})

# Converting columns to desired datatypes
Calendar_df['Date'] = pd.to_datetime(Calendar_df['Date'], dayfirst=True)
Calendar_df['StartingHour'] = pd.to_datetime(Calendar_df['StartingHour'], format='%H:%M').dt.time
Calendar_df['EndingHour'] = pd.to_datetime(Calendar_df['EndingHour'], format='%H:%M').dt.time

In [92]:
Calendar_df

,Date,StartingHour,EndingHour
0,2023-03-01,08:00:00,16:00:00
1,2023-03-02,08:00:00,16:00:00
2,2023-03-03,00:00:00,00:00:00
3,2023-03-04,08:00:00,16:00:00
4,2023-03-05,08:00:00,16:00:00
5,2023-03-06,08:00:00,16:00:00
6,2023-03-07,08:00:00,16:00:00
7,2023-03-08,08:00:00,16:00:00
8,2023-03-09,08:00:00,16:00:00
9,2023-03-10,00:00:00,00:00:00


In [93]:
import pandas as pd
from datetime import datetime

# Converting 'StartingHour' and 'EndingHour' to AM/PM string format
Calendar_df['StartingHour'] = Calendar_df['StartingHour'].apply(lambda x: x.strftime('%I:%M %p'))
Calendar_df['EndingHour'] = Calendar_df['EndingHour'].apply(lambda x: x.strftime('%I:%M %p'))

# Create temporary datetime objects for 'StartingHour' and 'EndingHour' to calculate 'WorkingHours'
Calendar_df['TempStart'] = pd.to_datetime(Calendar_df['StartingHour'], format='%I:%M %p')
Calendar_df['TempEnd'] = pd.to_datetime(Calendar_df['EndingHour'], format='%I:%M %p')

# Calculate WorkingHours and convert to total hours as numeric
Calendar_df['WorkingHours'] = ((Calendar_df['TempEnd'] - Calendar_df['TempStart']).dt.seconds) / 3600

# Drop the temporary columns
Calendar_df = Calendar_df.drop(columns=['TempStart', 'TempEnd'])

In [94]:
Calendar_df

,Date,StartingHour,EndingHour,WorkingHours
0,2023-03-01,08:00 AM,04:00 PM,8.0
1,2023-03-02,08:00 AM,04:00 PM,8.0
2,2023-03-03,12:00 AM,12:00 AM,0.0
3,2023-03-04,08:00 AM,04:00 PM,8.0
4,2023-03-05,08:00 AM,04:00 PM,8.0
5,2023-03-06,08:00 AM,04:00 PM,8.0
6,2023-03-07,08:00 AM,04:00 PM,8.0
7,2023-03-08,08:00 AM,04:00 PM,8.0
8,2023-03-09,08:00 AM,04:00 PM,8.0
9,2023-03-10,12:00 AM,12:00 AM,0.0


In [95]:
import numpy as np

Calendar_df['Type'] = np.where(Calendar_df['WorkingHours'] == 0, 'Non work', 'Standard')

In [96]:
Calendar_df

,Date,StartingHour,EndingHour,WorkingHours,Type
0,2023-03-01,08:00 AM,04:00 PM,8.0,Standard
1,2023-03-02,08:00 AM,04:00 PM,8.0,Standard
2,2023-03-03,12:00 AM,12:00 AM,0.0,Non work
3,2023-03-04,08:00 AM,04:00 PM,8.0,Standard
4,2023-03-05,08:00 AM,04:00 PM,8.0,Standard
5,2023-03-06,08:00 AM,04:00 PM,8.0,Standard
6,2023-03-07,08:00 AM,04:00 PM,8.0,Standard
7,2023-03-08,08:00 AM,04:00 PM,8.0,Standard
8,2023-03-09,08:00 AM,04:00 PM,8.0,Standard
9,2023-03-10,12:00 AM,12:00 AM,0.0,Non work


In [97]:
def excel_date_to_dt(date):
    return (pd.to_datetime('1900-01-01') + pd.to_timedelta(date - 2, unit='d')).date()

# Ensure 'DayNumber' column is numeric
Exceptions['DayNumber'] = pd.to_numeric(Exceptions['DayNumber'])

# Apply the function to 'DayNumber' column
Exceptions['Date'] = Exceptions['DayNumber'].apply(excel_date_to_dt)

# Convert 'Date' in Exceptions dataframe to datetime
Exceptions['Date'] = pd.to_datetime(Exceptions['Date'])

In [98]:
Exceptions

,DayNumber,StartingHour,EndingHour,Date
0,40179,0,0,2010-01-01
1,40329,0,0,2010-05-31
2,40364,0,0,2010-07-05
3,40427,0,0,2010-09-06
4,40507,0,0,2010-11-25
5,40536,0,0,2010-12-24
6,40543,0,0,2010-12-31
7,40693,0,0,2011-05-30
8,40728,0,0,2011-07-04
9,40791,0,0,2011-09-05


In [99]:
# Merge Calendar_df and Exceptions_df on Date:
df = pd.merge(Calendar_df, Exceptions, on='Date', how='left')

# Function to apply modifications
def modify_rows(row):
    if row['StartingHour_y'] == 0 and row['EndingHour_y'] == 0:
        row['StartingHour_x'] = '12:00 AM'
        row['EndingHour_x'] = '12:00 AM'
        row['WorkingHours'] = 0
        row['Type'] = 'Non work'
    return row

# Apply modifications
df = df.apply(modify_rows, axis=1)

# Drop unnecessary columns and rename columns
df = df.drop(columns=['StartingHour_y', 'EndingHour_y', 'DayNumber'])
df = df.rename(columns={'StartingHour_x':'StartingHour',
                        'EndingHour_x':'EndingHour'})

# The DataFrame 'df' now holds the data with all the required modifications

In [100]:
df

,Date,StartingHour,EndingHour,WorkingHours,Type
0,2023-03-01,08:00 AM,04:00 PM,8.0,Standard
1,2023-03-02,08:00 AM,04:00 PM,8.0,Standard
2,2023-03-03,12:00 AM,12:00 AM,0.0,Non work
3,2023-03-04,08:00 AM,04:00 PM,8.0,Standard
4,2023-03-05,08:00 AM,04:00 PM,8.0,Standard
5,2023-03-06,08:00 AM,04:00 PM,8.0,Standard
6,2023-03-07,08:00 AM,04:00 PM,8.0,Standard
7,2023-03-08,08:00 AM,04:00 PM,8.0,Standard
8,2023-03-09,08:00 AM,04:00 PM,8.0,Standard
9,2023-03-10,12:00 AM,12:00 AM,0.0,Non work


In [101]:
# Replace '0' with '00:00' and assign it back to the DataFrame
Exceptions['StartingHour'] = Exceptions['StartingHour'].replace('0', '00:00')
Exceptions['EndingHour'] = Exceptions['EndingHour'].replace('0', '00:00')

In [102]:
Exceptions

,DayNumber,StartingHour,EndingHour,Date
0,40179,00:00,00:00,2010-01-01
1,40329,00:00,00:00,2010-05-31
2,40364,00:00,00:00,2010-07-05
3,40427,00:00,00:00,2010-09-06
4,40507,00:00,00:00,2010-11-25
5,40536,00:00,00:00,2010-12-24
6,40543,00:00,00:00,2010-12-31
7,40693,00:00,00:00,2011-05-30
8,40728,00:00,00:00,2011-07-04
9,40791,00:00,00:00,2011-09-05


In [103]:
# Convert 'StartingHour' and 'EndingHour' columns to datetime
Exceptions['StartingHour'] = pd.to_datetime(Exceptions['StartingHour'], format='%H:%M').dt.strftime('%I:%M %p')
Exceptions['EndingHour'] = pd.to_datetime(Exceptions['EndingHour'], format='%H:%M').dt.strftime('%I:%M %p')

In [104]:
Exceptions

,DayNumber,StartingHour,EndingHour,Date
0,40179,12:00 AM,12:00 AM,2010-01-01
1,40329,12:00 AM,12:00 AM,2010-05-31
2,40364,12:00 AM,12:00 AM,2010-07-05
3,40427,12:00 AM,12:00 AM,2010-09-06
4,40507,12:00 AM,12:00 AM,2010-11-25
5,40536,12:00 AM,12:00 AM,2010-12-24
6,40543,12:00 AM,12:00 AM,2010-12-31
7,40693,12:00 AM,12:00 AM,2011-05-30
8,40728,12:00 AM,12:00 AM,2011-07-04
9,40791,12:00 AM,12:00 AM,2011-09-05


In [105]:
# Create temporary DataFrames which parses the time columns to datetime
temp_start = pd.to_datetime(Exceptions['StartingHour'].astype(str), format='%I:%M %p')
temp_end = pd.to_datetime(Exceptions['EndingHour'].astype(str), format='%I:%M %p')

# Perform the subtraction Operation and convert it to seconds, then divide by 3600 to get hours
Exceptions['WorkingHours'] = (temp_end - temp_start).dt.total_seconds() / 3600

In [106]:
Exceptions

,DayNumber,StartingHour,EndingHour,Date,WorkingHours
0,40179,12:00 AM,12:00 AM,2010-01-01,0.0
1,40329,12:00 AM,12:00 AM,2010-05-31,0.0
2,40364,12:00 AM,12:00 AM,2010-07-05,0.0
3,40427,12:00 AM,12:00 AM,2010-09-06,0.0
4,40507,12:00 AM,12:00 AM,2010-11-25,0.0
5,40536,12:00 AM,12:00 AM,2010-12-24,0.0
6,40543,12:00 AM,12:00 AM,2010-12-31,0.0
7,40693,12:00 AM,12:00 AM,2011-05-30,0.0
8,40728,12:00 AM,12:00 AM,2011-07-04,0.0
9,40791,12:00 AM,12:00 AM,2011-09-05,0.0


In [107]:
Exceptions_zero_hours = Exceptions[Exceptions['WorkingHours'] == 0]
for idx, row in Exceptions_zero_hours.iterrows():
    date = row['Date']
    if date in Calendar_df['Date'].values:
        Calendar_df.loc[Calendar_df['Date'] == date, 'StartingHour'] = row['StartingHour']
        Calendar_df.loc[Calendar_df['Date'] == date, 'EndingHour'] = row['EndingHour']
        Calendar_df.loc[Calendar_df['Date'] == date, 'WorkingHours'] = row['WorkingHours']
        Calendar_df.loc[Calendar_df['Date'] == date, 'Type'] = "Non work"

In [108]:
Calendar_df

,Date,StartingHour,EndingHour,WorkingHours,Type
0,2023-03-01,08:00 AM,04:00 PM,8.0,Standard
1,2023-03-02,08:00 AM,04:00 PM,8.0,Standard
2,2023-03-03,12:00 AM,12:00 AM,0.0,Non work
3,2023-03-04,08:00 AM,04:00 PM,8.0,Standard
4,2023-03-05,08:00 AM,04:00 PM,8.0,Standard
5,2023-03-06,08:00 AM,04:00 PM,8.0,Standard
6,2023-03-07,08:00 AM,04:00 PM,8.0,Standard
7,2023-03-08,08:00 AM,04:00 PM,8.0,Standard
8,2023-03-09,08:00 AM,04:00 PM,8.0,Standard
9,2023-03-10,12:00 AM,12:00 AM,0.0,Non work


In [109]:
# Filter rows in 'Exceptions' where 'WorkingHours' is not equal to 0
exceptions_nonzero_hours = Exceptions[Exceptions['WorkingHours'] != 0]

# Find corresponding rows in 'Calendar_df'
for index, row in exceptions_nonzero_hours.iterrows():
    date = row['Date']

    # If 'Calendar_df' contains the date
    if date in Calendar_df['Date'].values:
        # Update corresponding 'StartingHour', 'EndingHour', 'WorkingHours and 'Type' in 'Calendar_df'
        Calendar_df.loc[Calendar_df['Date'] == date, 'StartingHour'] = row['StartingHour']
        Calendar_df.loc[Calendar_df['Date'] == date, 'EndingHour'] = row['EndingHour']
        Calendar_df.loc[Calendar_df['Date'] == date, 'WorkingHours'] = row['WorkingHours']
        Calendar_df.loc[Calendar_df['Date'] == date, 'Type'] = "Exception"

In [110]:
Calendar_df

,Date,StartingHour,EndingHour,WorkingHours,Type
0,2023-03-01,08:00 AM,04:00 PM,8.0,Standard
1,2023-03-02,08:00 AM,04:00 PM,8.0,Standard
2,2023-03-03,12:00 AM,12:00 AM,0.0,Non work
3,2023-03-04,08:00 AM,04:00 PM,8.0,Standard
4,2023-03-05,08:00 AM,04:00 PM,8.0,Standard
5,2023-03-06,08:00 AM,04:00 PM,8.0,Standard
6,2023-03-07,08:00 AM,04:00 PM,8.0,Standard
7,2023-03-08,08:00 AM,04:00 PM,8.0,Standard
8,2023-03-09,08:00 AM,04:00 PM,8.0,Standard
9,2023-03-10,12:00 AM,12:00 AM,0.0,Non work


In [111]:
# Assuming Calendar_df is your DataFrame
calendar_file_path = 'HousingSTP-Calendar.xlsx'  # you may want to include full path here
with pd.ExcelWriter(calendar_file_path, engine='openpyxl') as writer:
    Calendar_df.to_excel(writer, sheet_name='Calendar', index=False)